In [1]:
##%%
# A/B Test Analysis: ARPI_D1 and D1 Retention

import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt

# Data
n_control = 10000
n_test = 10000

revenue_control = 4500
revenue_test = 4800

retained_control = 3200
retained_test = 3500

# 1. Calculate ARPI_D1 and D1 Retention rates
arpi_control = revenue_control / n_control
arpi_test = revenue_test / n_test

retention_control = retained_control / n_control
retention_test = retained_test / n_test

print(f"ARPI_D1: Control = {arpi_control:.4f}, Test = {arpi_test:.4f}")
print(f"D1 Retention: Control = {retention_control:.4f}, Test = {retention_test:.4f}")

# 2. Statistical significance tests

# a) ARPI_D1: Two-sample t-test (assuming normality and equal n)
# Assume standard deviation (sigma) is unknown, so use pooled std estimate
# For illustration, assume std = 1 for both groups (since not provided)
std_arpi = 1.0
se_arpi = np.sqrt(std_arpi**2/n_control + std_arpi**2/n_test)
t_stat_arpi = (arpi_test - arpi_control) / se_arpi
p_value_arpi = 2 * (1 - stats.norm.cdf(np.abs(t_stat_arpi)))

print(f"\nARPI_D1 t-statistic: {t_stat_arpi:.2f}, p-value: {p_value_arpi:.4f}")

# b) D1 Retention: Two-proportion z-test
p1 = retention_control
p2 = retention_test
p_pool = (retained_control + retained_test) / (n_control + n_test)
se_ret = np.sqrt(p_pool * (1 - p_pool) * (1/n_control + 1/n_test))
z_stat_ret = (p2 - p1) / se_ret
p_value_ret = 2 * (1 - stats.norm.cdf(np.abs(z_stat_ret)))

print(f"D1 Retention z-statistic: {z_stat_ret:.2f}, p-value: {p_value_ret:.4f}")

# 3. Power calculation

# a) ARPI_D1 power
effect_size_arpi = (arpi_test - arpi_control) / std_arpi
alpha = 0.05
power_arpi = stats.norm.cdf(effect_size_arpi/np.sqrt(2) - stats.norm.ppf(1-alpha/2))
print(f"\nARPI_D1 effect size: {effect_size_arpi:.3f}, Power: {power_arpi:.3f}")

# b) D1 Retention power
# Cohen's h for proportions
def cohens_h(p1, p2):
    return 2 * (np.arcsin(np.sqrt(p2)) - np.arcsin(np.sqrt(p1)))

h = cohens_h(p1, p2)
# Power for two-proportion z-test
from statsmodels.stats.power import NormalIndPower
power_analysis = NormalIndPower()
power_ret = power_analysis.power(effect_size=h, nobs1=n_control, alpha=alpha, ratio=1.0, alternative='two-sided')
print(f"D1 Retention Cohen's h: {h:.3f}, Power: {power_ret:.3f}")

# 4. Summary and Recommendation
from IPython.display import display, Markdown

summary = f"""
### Summary

- **ARPI_D1:** Control = {arpi_control:.4f}, Test = {arpi_test:.4f}, p-value = {p_value_arpi:.4f}, Power = {power_arpi:.3f}
- **D1 Retention:** Control = {retention_control:.4f}, Test = {retention_test:.4f}, p-value = {p_value_ret:.4f}, Power = {power_ret:.3f}

#### Recommendation

- **Statistical Significance:** 
  - ARPI_D1: {"Significant" if p_value_arpi < 0.05 else "Not significant"}
  - D1 Retention: {"Significant" if p_value_ret < 0.05 else "Not significant"}
- **Power:** 
  - ARPI_D1: {"Sufficient" if power_arpi > 0.8 else "Insufficient"}
  - D1 Retention: {"Sufficient" if power_ret > 0.8 else "Insufficient"}

- **Should the feature be rolled out?**
  - {"Yes, both metrics show significant improvement and sufficient power." if (p_value_arpi < 0.05 and p_value_ret < 0.05 and power_arpi > 0.8 and power_ret > 0.8) else "No, more data or further testing is needed."}

- **Is there enough evidence?**
  - {"Yes" if (p_value_arpi < 0.05 and p_value_ret < 0.05 and power_arpi > 0.8 and power_ret > 0.8) else "No"}

- **If not significant, what next?**
  - Increase sample size or run the test longer to achieve sufficient power and statistical significance.

"""

display(Markdown(summary))
##%%

ARPI_D1: Control = 0.4500, Test = 0.4800
D1 Retention: Control = 0.3200, Test = 0.3500

ARPI_D1 t-statistic: 2.12, p-value: 0.0339
D1 Retention z-statistic: 4.49, p-value: 0.0000

ARPI_D1 effect size: 0.030, Power: 0.026
D1 Retention Cohen's h: 0.064, Power: 0.994



### Summary

- **ARPI_D1:** Control = 0.4500, Test = 0.4800, p-value = 0.0339, Power = 0.026
- **D1 Retention:** Control = 0.3200, Test = 0.3500, p-value = 0.0000, Power = 0.994

#### Recommendation

- **Statistical Significance:** 
  - ARPI_D1: Significant
  - D1 Retention: Significant
- **Power:** 
  - ARPI_D1: Insufficient
  - D1 Retention: Sufficient

- **Should the feature be rolled out?**
  - No, more data or further testing is needed.

- **Is there enough evidence?**
  - No

- **If not significant, what next?**
  - Increase sample size or run the test longer to achieve sufficient power and statistical significance.

